In [3]:
import tensorflow as tf
import PIL
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)
from tensorflow.keras.applications import mobilenet as _mobilenet
from keras_tqdm import TQDMNotebookCallback
import random

AttributeError: module 'numpy' has no attribute 'integer'

In [ ]:
import numpy as np
def load_image(img_file, target_size=(224,224)):
    X = np.zeros((1, *target_size, 3))
    X[0, ] = np.asarray(tf.keras.preprocessing.image.load_img(
        img_file, 
        target_size=target_size)
    )
    X = tf.keras.applications.mobilenet.preprocess_input(X)
    return X

import os
def ensure_folder_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet(
# The 3 is the three dimensions of the input: r,g,b.
  input_shape=(224, 224, 3), 
  include_top=False, 
  pooling='avg'
)


In [ ]:
Trump_image_id = os.listdir('/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image/trump')[0]
Trump_image = load_image(os.path.join('/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image/trump',Trump_image_id))
model.predict(Trump_image).shape

In [ ]:
x = Dropout(rate=0.4)(model.output)
# This determines the number of outputs, and thus the number of categories, in my test case 2: Trump or Boris
x = Dense(2)(x)
x = Softmax()(x)
model= Model(model.inputs, x)

In [ ]:
for layer in model.layers[:-3]:
    layer.trainable = False


In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(lr=0.001),
    loss='categorical_crossentropy'
)


In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=_mobilenet.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

ensure_folder_exists('training_aug')
training = datagen.flow_from_directory( 
    '/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image', 
    target_size=(224, 224),
    save_to_dir='/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/training_aug',
    subset='training'
) 

ensure_folder_exists('validation_aug')
validation = datagen.flow_from_directory( 
    '/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image',
    target_size=(224, 224),
    save_to_dir='/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/validation_aug',
    subset='validation'
)

In [ ]:
batch_size = 32

history = model.fit_generator(
    generator=training,
    steps_per_epoch=training.samples // batch_size,
    epochs=10,
    validation_data=validation,
    validation_steps=validation.samples // batch_size
)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
random_Boris = random.choice(os.listdir('/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image/boris'))
Boris_path = os.path.join('/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image/boris',random_Boris)
Boris = load_image(Boris_path)
random_Trump = random.choice(os.listdir('/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image/trump'))
Trump_path = os.path.join('/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/image/trump',random_Trump)
Trump = load_image(Trump_path)

In [ ]:
tf.keras.preprocessing.image.load_img(Boris_path, target_size=(224,224))

In [ ]:
tf.keras.preprocessing.image.load_img(Trump_path, target_size=(224,224))

In [ ]:
print(f'''Boris:{np.round(model.predict(Boris),2)}''')

In [ ]:
print(f'''Trump:{np.round(model.predict(Trump),2)}''')

In [ ]:
Barron_path = '/Users/guoyunyun/Desktop/YunyunGuo_RC11_Python/Wanting-Ding---Yunyun-Guo/TrumpBoris/Barron.jpg'
Barron = load_image(Barron_path)

In [ ]:
tf.keras.preprocessing.image.load_img(Barron_path, target_size=(224,224))

In [ ]:
print(f'''Barron:{np.round(model.predict(Barron),2)}''')